**Consolidated Monthly Rapport**

This code generates the Monthly Rapport

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

In [2]:
# Define the file path for Shipments Out
# Replace the path with the actual path to your Shipments Consolidated file
shipments = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Shipments.xlsx')

# Define the file path for Goods In
# Replace the path with the actual path to your Inbound Orderline file
inbound = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Inbound Order Line.xlsx')

# Define the file path for Moved to Production
# Replace the path with the actual path to your Production file
production = pd.read_csv(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\Production_Consolidated.csv')

# Define the file path for Inbound from Production
# Replace the path with the actual path to your Inbound from Production file
inbound_prod = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Inbound from Production.xlsx')

# Define the file path for Inventory Ultimo
# Replace the path with the actual path to your Inventory Consolidated file
inventory_skandflen = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Inventory Skanderborg Flensborg.xlsx')
inventory_handewitt = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) Aug Final Inventory Handewitt.xlsx')

# Define the file path for Current Primo
# Replace the path with the actual path to your previous month's DK Rapport file
previous_rapport_DK = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) July 2025 Rapport.xlsx', sheet_name='DK')
# Replace the path with the actual path to your previous month's DE Rapport file
previous_rapport_DE = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) July 2025 Rapport.xlsx', sheet_name='DE')
# Replace the path with the actual path to your previous month's DE Rapport file
previous_rapport_TOT = pd.read_excel(r'C:\Users\USER\OneDrive\Documents\House of Analytics\ITA\-Ita-Data_Cleanup\(ITA) July 2025 Rapport.xlsx', sheet_name='Total')

Data Transformation - Aggregations: Inventory

In [3]:
# Cleanup files
# Cleanup the Inventory files
inventory_skandflen['Område'] = np.where(inventory_skandflen['Område'].isin(['S2', 'S4']), 'DK', np.where(inventory_skandflen['Område'] == 'F1', 'DE', inventory_skandflen['Område']))
inventory_handewitt['Område'] = 'DE'

# Aggregate the total Mængde per Varenummer and Område
inventory_skandflen_grouped = inventory_skandflen.groupby(['Varenummer', 'Område'], as_index=False)['Mængde'].sum()

# Remove #FP from Varenummer in inventory_skandflen_grouped
inventory_skandflen_grouped['Varenummer'] = inventory_skandflen_grouped['Varenummer'].str.replace('#FP', '', regex=False)

In [4]:
# Aggregate the total On Hand per SKU and Område from Handewitt
inventory_handewitt_grouped = inventory_handewitt.groupby(['SKU', 'Område'], as_index=False)['On Hand'].sum()
inventory_handewitt_grouped.rename(columns={'SKU': 'Varenummer', 'On Hand': 'Mængde'}, inplace=True)
inventory_handewitt_grouped.head()


,Varenummer,Område,Mængde
0,6429,DE,0
1,6430,DE,0
2,6431,DE,0
3,6432,DE,0
4,6436,DE,0


In [5]:
# Merge both inventory dataframes and establish the variable Inventroy Ultimo
final_inventory = pd.concat([inventory_skandflen_grouped, inventory_handewitt_grouped], ignore_index=True)
final_inventory = final_inventory.groupby(['Varenummer', 'Område'], as_index=False)['Mængde'].sum()
final_inventory.head()

,Varenummer,Område,Mængde
0,6429,DE,0
1,6430,DE,0
2,6431,DE,0
3,6432,DE,0
4,6436,DE,0


In [6]:
# Rename columns for merging
final_inventory.rename(columns={'Varenummer': 'SKU', 'Mængde': 'Inventory ULTIMO'}, inplace=True)

# Split data to DK and DE for merging with previous rapport files
INVENTORY_DK = final_inventory[final_inventory['Område'] == 'DK']
INVENTORY_DE = final_inventory[final_inventory['Område'] == 'DE']
INVENTORY_DK = INVENTORY_DK[['SKU', 'Inventory ULTIMO']]
INVENTORY_DE = INVENTORY_DE[['SKU', 'Inventory ULTIMO']]

Data Transformation - Aggregations: Ultimo and Primo

In [7]:
## Creating the two new variables: Previous Month Ultimo and Current Primo
# Remove "#FP" from SKU in July Rapport files
previous_rapport_DK['SKU'] = previous_rapport_DK['SKU'].str.replace('#FP', '', regex=False)
previous_rapport_DE['SKU'] = previous_rapport_DE['SKU'].str.replace('#FP', '', regex=False)
previous_rapport_TOT['SKU'] = previous_rapport_TOT['SKU'].str.replace('#FP', '', regex=False)

# Extract the Current DK Primo using the previous month's "Inventroy ULTIMO"
ULTIMO_DK = previous_rapport_DK.groupby('SKU', as_index=False)['Inventroy ULTIMO'].sum()

# Extract the Current DE Primo using the previous month's "Inventroy ULTIMO"
ULTIMO_DE = previous_rapport_DE.groupby('SKU', as_index=False)['Inventroy ULTIMO'].sum()

# Extract the Current TOT Primo using the previous month's "Inventroy ULTIMO"
ULTIMO_TOT = previous_rapport_TOT.groupby('SKU', as_index=False)['Inventroy ULTIMO'].sum()
ULTIMO_TOT.head()

,SKU,Inventroy ULTIMO
0,###-2XL-B,10
1,###-2XL-W,8
2,###-L-B,19
3,###-L-W,19
4,###-M-B,20


In [8]:
## Reorganize DK Ultimo and Primo variables
# Rename ULTIMO_DK to Previous Month Primo
ULTIMO_DK.rename(columns={'Inventroy ULTIMO': 'Previous Month Ultimo'}, inplace=True)

# Create new column 'Område' and set its value to 'DK'
ULTIMO_DK['Område'] = 'DK'

# Create new column 'Current Month Primo' and set its value to the same as 'Previous Month Ultimo'
ULTIMO_DK['Current Month Primo'] = ULTIMO_DK['Previous Month Ultimo']

# Reorder columns to have 'SKU', 'Område', 'Previous Month Primo', 'Current Month Primo'
ULTIMO_DK = ULTIMO_DK[['SKU', 'Område', 'Previous Month Ultimo', 'Current Month Primo']]

In [9]:
## Reorganize DE Ultimo and Primo variables
# Rename ULTIMO_DE to Previous Month Primo
ULTIMO_DE.rename(columns={'Inventroy ULTIMO': 'Previous Month Ultimo'}, inplace=True)

# Create new column 'Område' and set its value to 'DE'
ULTIMO_DE['Område'] = 'DE'

# Create new column 'Current Month Primo' and set its value to the same as 'Previous Month Primo'
ULTIMO_DE['Current Month Primo'] = ULTIMO_DE['Previous Month Ultimo']

# Reorder columns to have 'SKU', 'Område', 'Previous Month Primo', 'Current Month Primo'
ULTIMO_DE = ULTIMO_DE[['SKU', 'Område', 'Previous Month Ultimo', 'Current Month Primo']]

In [10]:
## Create 'TOT Ultimo' by adding DK and DE 'Previous Month Ultimo' and 'Current Month Primo' based on SKU
ULTIMO_TOT = pd.merge(ULTIMO_DK, ULTIMO_DE, on='SKU', how='outer', suffixes=('_DK', '_DE'))
ULTIMO_TOT['Previous Month Ultimo'] = ULTIMO_TOT['Previous Month Ultimo_DK'].fillna(0) + ULTIMO_TOT['Previous Month Ultimo_DE'].fillna(0)
ULTIMO_TOT['Current Month Primo'] = ULTIMO_TOT['Current Month Primo_DK'].fillna(0) + ULTIMO_TOT['Current Month Primo_DE'].fillna(0)
ULTIMO_TOT = ULTIMO_TOT[['SKU', 'Previous Month Ultimo', 'Current Month Primo']]
ULTIMO_TOT.head()


,SKU,Previous Month Ultimo,Current Month Primo
0,###-2XL-B,10.0,10.0
1,###-2XL-W,8.0,8.0
2,###-L-B,19.0,19.0
3,###-L-W,19.0,19.0
4,###-M-B,20.0,20.0


In [11]:
## Creating new variable: Goods In
# Cleanup SKU by removeing "#FP"
inbound['Varenummer'] = inbound['Varenummer'].str.replace('#FP', '', regex=False)

# Create 'Goods in' by aggregating the total Mængde modtaget per Varenummer and Område filtering by 'DK' only
goods_in_DK = inbound[inbound['Område'] == 'DK']
goods_in_DK_grouped = goods_in_DK.groupby(['Varenummer', 'Område'], as_index=False)['Mængde modtaget'].sum()
goods_in_DK_grouped.rename(columns={'Varenummer': 'SKU', 'Mængde modtaget': 'Goods In'}, inplace=True)
goods_in_DK_grouped = goods_in_DK_grouped[['SKU', 'Goods In', 'Område']]

# Create 'Goods in' by aggregating the total Mængde modtaget per Varenummer and Område filtering by 'DE' only
goods_in_DE = inbound[inbound['Område'] == 'DE']
goods_in_DE_grouped = goods_in_DE.groupby(['Varenummer', 'Område'], as_index=False)['Mængde modtaget'].sum()
goods_in_DE_grouped.rename(columns={'Varenummer': 'SKU', 'Mængde modtaget': 'Goods In'}, inplace=True)
goods_in_DE_grouped = goods_in_DE_grouped[['SKU', 'Goods In', 'Område']]

# Create 'Goods in' by aggregating the total Mængde modtaget per Varenummer and Område by adding DK and DE values to get 'TOT'
goods_in_TOT = pd.merge(goods_in_DK_grouped, goods_in_DE_grouped, on='SKU', how='outer', suffixes=('_DK', '_DE'))
goods_in_TOT['Goods In'] = goods_in_TOT['Goods In_DK'].fillna(0) + goods_in_TOT['Goods In_DE'].fillna(0)
goods_in_TOT = goods_in_TOT[['SKU', 'Goods In']]


In [12]:
## Creating new variable: Moved to Production (Negative Values)
# Cleanup SKU by removeing "#FP"
production['ItemNumber'] = production['ItemNumber'].str.replace('#FP', '', regex=False)

# Create 'Moved to Production' by aggregating the total Quantity Produced per SKU filtering SourceCountry by 'DK' only
production_DK = production[production['SourceCountry'] == 'DK']
moved_to_production_DK = production_DK.groupby(['ItemNumber','SourceCountry'], as_index=False)['QuantityProduced'].sum()
moved_to_production_DK.rename(columns={'QuantityProduced': 'Moved to Production'}, inplace=True)
moved_to_production_DK = moved_to_production_DK[['ItemNumber', 'Moved to Production', 'SourceCountry']]
moved_to_production_DK['Moved to Production'] = moved_to_production_DK['Moved to Production'] * -1

# Create 'Moved to Production' by aggregating the total Quantity Produced per SKU filtering SourceCountry by 'DE' only
production_DE = production[production['SourceCountry'] == 'DE']
moved_to_production_DE = production_DE.groupby(['ItemNumber','SourceCountry'], as_index=False)['QuantityProduced'].sum()
moved_to_production_DE.rename(columns={'QuantityProduced': 'Moved to Production'}, inplace=True)
moved_to_production_DE = moved_to_production_DE[['ItemNumber', 'Moved to Production', 'SourceCountry']]
moved_to_production_DE['Moved to Production'] = moved_to_production_DE['Moved to Production'] * -1

# Create 'Moved to Production' by aggregating the total Quantity Produced per SKU by combining DK and DE values to get 'TOT'
moved_to_production_TOT = pd.merge(moved_to_production_DK, moved_to_production_DE, on='ItemNumber', how='outer', suffixes=('_DK', '_DE'))
moved_to_production_TOT['Moved to Production'] = moved_to_production_TOT['Moved to Production_DK'].fillna(0) + moved_to_production_TOT['Moved to Production_DE'].fillna(0)
moved_to_production_TOT = moved_to_production_TOT[['ItemNumber', 'Moved to Production', 'SourceCountry_DK', 'SourceCountry_DE']]


In [13]:
## Creating new variable: Inbound from Production
# Cleanup SKU by removeing "#FP"
inbound_prod['Varenummer'] = inbound_prod['Varenummer'].str.replace('#FP', '', regex=False)
# Create 'Inbound from Production' by aggregating the total Mængde modtaget per Varenummer and Område filtering by 'DK' only
inbound_prod_DK = inbound_prod[inbound_prod['Country'] == 'DK']
inbound_prod_DK_grouped = inbound_prod_DK.groupby(['Varenummer', 'Country'], as_index=False)['Mængde modtaget'].sum()
inbound_prod_DK_grouped.rename(columns={'Varenummer': 'SKU', 'Mængde modtaget': 'Inbound from Production'}, inplace=True)
inbound_prod_DK_grouped = inbound_prod_DK_grouped[['SKU', 'Inbound from Production', 'Country']]

# Create 'Inbound from Production' by aggregating the total Mængde modtaget per Varenummer and Område filtering by 'DE' only
inbound_prod_DE = inbound_prod[inbound_prod['Country'] == 'DE']
inbound_prod_DE_grouped = inbound_prod_DE.groupby(['Varenummer', 'Country'], as_index=False)['Mængde modtaget'].sum()
inbound_prod_DE_grouped.rename(columns={'Varenummer': 'SKU', 'Mængde modtaget': 'Inbound from Production'}, inplace=True)
inbound_prod_DE_grouped = inbound_prod_DE_grouped[['SKU', 'Inbound from Production', 'Country']]

# Create 'Inbound from Production' by aggregating the total Mængde modtaget per Varenummer and Område by adding DK and DE values to get 'TOT'
inbound_prod_TOT = pd.merge(inbound_prod_DK_grouped, inbound_prod_DE_grouped, on='SKU', how='outer', suffixes=('_DK', '_DE'))
inbound_prod_TOT['Inbound from Production'] = inbound_prod_TOT['Inbound from Production_DK'].fillna(0) + inbound_prod_TOT['Inbound from Production_DE'].fillna(0)
inbound_prod_TOT = inbound_prod_TOT[['SKU', 'Inbound from Production']]

inbound_prod_TOT.head()

,SKU,Inbound from Production
0,39473090-C-DE,126.0
1,39473090-C-DK,80.0
2,A920Pro-2AW-RE6-20EU-C-FR,1840.0
3,A920Pro-2AW-RE6-20EU-C-UK,200.0
4,Cposx5-C-DE,954.0


In [14]:
## Creating new variable: Shipments Out (Negative Values)
# Cleanup SKU by removeing "#FP"
shipments['ItemNumber'] = shipments['ItemNumber'].str.replace('#FP', '', regex=False)

# Create 'Shipments Out' by aggregating the total Quantity Shipped per SKU filtering SourceCountry by 'DK' only
shipments_DK = shipments[shipments['PickingWarehouseCountry'] == 'DK']
shipments_DK_grouped = shipments_DK.groupby(['ItemNumber', 'PickingWarehouseCountry'], as_index=False)['QuantityPacked'].sum()
shipments_DK_grouped.rename(columns={'ItemNumber': 'SKU', 'QuantityPacked': 'Shipments Out'}, inplace=True)
shipments_DK_grouped = shipments_DK_grouped[['SKU', 'Shipments Out', 'PickingWarehouseCountry']]
shipments_DK_grouped['Shipments Out'] = shipments_DK_grouped['Shipments Out'] * -1

shipments_DK_grouped.rename(columns={'ItemNumber': 'SKU'}, inplace=True)

# Create 'Shipments Out' by aggregating the total Quantity Shipped per SKU filtering SourceCountry by 'DE' only
shipments_DE = shipments[shipments['PickingWarehouseCountry'] == 'DE']
shipments_DE_grouped = shipments_DE.groupby(['ItemNumber', 'PickingWarehouseCountry'], as_index=False)['QuantityPacked'].sum()
shipments_DE_grouped.rename(columns={'ItemNumber': 'SKU', 'QuantityPacked': 'Shipments Out'}, inplace=True)
shipments_DE_grouped = shipments_DE_grouped[['SKU', 'Shipments Out', 'PickingWarehouseCountry']]
shipments_DE_grouped['Shipments Out'] = shipments_DE_grouped['Shipments Out'] * -1

shipments_DE_grouped.rename(columns={'ItemNumber': 'SKU'}, inplace=True)

# Create 'Shipments Out' by aggregating the total Quantity Shipped per SKU by combining DK and DE values to get 'TOT'
shipments_TOT = pd.merge(shipments_DK_grouped, shipments_DE_grouped, on='SKU', how='outer', suffixes=('_DK', '_DE'))
shipments_TOT['Shipments Out'] = shipments_TOT['Shipments Out_DK'].fillna(0) + shipments_TOT['Shipments Out_DE'].fillna(0)
shipments_TOT = shipments_TOT[['SKU', 'Shipments Out', 'PickingWarehouseCountry_DK', 'PickingWarehouseCountry_DE']]

shipments_TOT.head()

,SKU,Shipments Out,PickingWarehouseCountry_DK,PickingWarehouseCountry_DE
0,###-2XL-B,-2.0,DK,NaN
1,###-L-B,-5.0,DK,NaN
2,###-L-W,-12.0,DK,NaN
3,###-M-B,-3.0,DK,NaN
4,###-M-W,-16.0,DK,NaN


In [ ]:
## Creating the variable: Variants and Combining all Sheets according to location: DK, DE and TOT
# Create variants by subtracting Inventory Ultimo by the SUM of all combined Current Month Primo, Goods In, Moved to Production, Inbound from Production, Shipments Out
# Merge all dataframes into a single dataframe based on SKU filtering by DK only
merged_DK = pd.merge(ULTIMO_DK[['SKU', 'Previous Month Ultimo', 'Current Month Primo']], INVENTORY_DK, on='SKU', how='outer')
merged_DK = pd.merge(merged_DK, goods_in_DK_grouped[['SKU', 'Goods In']], on='SKU', how='outer')
merged_DK = pd.merge(merged_DK, moved_to_production_DK[['ItemNumber', 'Moved to Production']], left_on='SKU', right_on='ItemNumber', how='outer')
merged_DK = pd.merge(merged_DK, inbound_prod_DK_grouped[['SKU', 'Inbound from Production']], on='SKU', how='outer')
merged_DK = pd.merge(merged_DK, shipments_DK_grouped[['SKU', 'Shipments Out']], on='SKU', how='outer')
merged_DK['Variants'] = merged_DK['Inventory ULTIMO'].fillna(0) - (merged_DK['Current Month Primo'].fillna(0) + 
                                                                   merged_DK['Goods In'].fillna(0) + 
                                                                   merged_DK['Moved to Production'].fillna(0) + 
                                                                   merged_DK['Inbound from Production'].fillna(0) + 
                                                                   merged_DK['Shipments Out'].fillna(0))

merged_DK = merged_DK[['SKU', 'Previous Month Ultimo', 'Current Month Primo', 'Goods In', 'Moved to Production', 'Inbound from Production', 'Shipments Out', 'Variants', 'Inventory ULTIMO']]

# Merge all dataframes into a single dataframe based on SKU filtering by DE only
merged_DE = pd.merge(ULTIMO_DE[['SKU', 'Previous Month Ultimo', 'Current Month Primo']], INVENTORY_DE, on='SKU', how='outer')
merged_DE = pd.merge(merged_DE, goods_in_DE_grouped[['SKU', 'Goods In']], on='SKU', how='outer')
merged_DE = pd.merge(merged_DE, moved_to_production_DE[['ItemNumber', 'Moved to Production']], left_on='SKU', right_on='ItemNumber', how='outer')
merged_DE = pd.merge(merged_DE, inbound_prod_DE_grouped[['SKU', 'Inbound from Production']], on='SKU', how='outer')
merged_DE = pd.merge(merged_DE, shipments_DE_grouped[['SKU', 'Shipments Out']], on='SKU', how='outer')
merged_DE['Variants'] = merged_DE['Inventory ULTIMO'].fillna(0) - (merged_DE['Current Month Primo'].fillna(0) + 
                                                                   merged_DE['Goods In'].fillna(0) +
                                                                    merged_DE['Moved to Production'].fillna(0) +
                                                                     merged_DE['Inbound from Production'].fillna(0) +
                                                                        merged_DE['Shipments Out'].fillna(0))

merged_DE = merged_DE[['SKU', 'Previous Month Ultimo', 'Current Month Primo', 'Goods In', 'Moved to Production', 'Inbound from Production', 'Shipments Out', 'Variants', 'Inventory ULTIMO']]

## Separator ##
Just need to combine all the values together